In [9]:
#setup the environment
#login using az login
# must use gpt-4 with the deployment API of 1106-Preview or later
import os
import json
from openai import AzureOpenAI
from azure.core.exceptions import AzureError
import sys
import requests
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
apiKey = input("Enter your Azure OpenAI API Key: ")
apiEndpoint = input("Enter your Azure OpenAI API Endpoint: ")
model = input("Enter your model deployment name: ")
useFunction = True

client = AzureOpenAI(
    api_key = apiKey,  
    api_version = "2023-12-01-preview",
    azure_endpoint = apiEndpoint
)



In [10]:
def get_current_weather(obj={}):
    location = obj.get('location')
    try:
        unit = obj.get('unit')
    except:
        unit = "Fahrenheit"
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    
print(get_current_weather({'location': 'Tokyo'}))

{"location": "Tokyo", "temperature": "10", "unit": null}


In [11]:

def get_lat_long_weather(obj={}):
    latitude = str(obj.get('latitude'))
    longitude = str(obj.get('longitude'))
    response = requests.get(f'https://api.weather.gov/points/{latitude},{longitude}')
    unit="fahrenheit"
    if response.status_code == 200:
         # Parse the response as JSON
        data = response.json()
        #print(data)
        office = data.get("properties",{}).get("gridId")
        gridx = data.get("properties",{}).get("gridX")
        gridy = data.get("properties",{}).get("gridY")

        #print(office)
        #print(gridx)
        #print(gridy)
        # Make a request to the /gridpoints/{office}/{gridX},{gridY}/forecast endpoint
        response = requests.get(f'https://api.weather.gov/gridpoints/{office}/{gridx},{gridy}/forecast')
        if response.status_code == 200:
            # Parse the response as JSON
            data = response.json()
            #print(data)
            # Get the current temperature
            temperature = data.get("properties",{}).get("periods", [])[0].get("temperature")
            detailedForecast = data.get("properties",{}).get("periods", [])[0].get("detailedForecast")
            name = data.get("properties",{}).get("periods", [])[0].get("name")
            #print(temperature)
            # Return the temperature
            return json.dumps({"latitude": latitude, "longitude": longitude, 
                               "temperature": temperature, "unit": unit, 
                               "detailedForecast": detailedForecast,"name": name })
        else:
            print(f"Request failed with status code step 2 {response.status_code}")

    else:
        print(f"Request failed with status code step 1 {response.status_code}")


print("### get_lat_long_weather ###")
print( get_lat_long_weather( {'latitude':38.8304, 'longitude':-77.1964} ))

### get_lat_long_weather ###
{"latitude": "38.8304", "longitude": "-77.1964", "temperature": 55, "unit": "fahrenheit", "detailedForecast": "Sunny, with a high near 55. Northwest wind 22 to 25 mph, with gusts as high as 47 mph.", "name": "This Afternoon"}


In [18]:

def user_information(obj={}):
    # Define the scope for Microsoft Graph API
    scope = "https://graph.microsoft.com/.default"

    # Get a credential object using the Azure Identity library
    credential = DefaultAzureCredential()

    # Get an access token
    token: AccessToken = credential.get_token(scope)

    # Define the endpoint URL
    url = 'https://graph.microsoft.com/v1.0/me'

    # Define headers for the request
    headers = {
        'Authorization': 'Bearer ' + token.token,
        'Content-Type': 'application/json'
    }

    # Send the GET request
    response = requests.get(url, headers=headers)

    # Check the response
    if response.status_code == 200:
        data = response.json()
        return json.dumps(data, indent=4)
    else:
        print(f"Request failed with status code {response.status_code}")

print("### user_information ###")
print(user_information())

### user_information ###
{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",
    "businessPhones": [],
    "displayName": "Patrick Davis",
    "givenName": "Patrick",
    "jobTitle": "SR TECHNICAL SPECIALIST",
    "mail": "patrickdavis@microsoft.com",
    "mobilePhone": null,
    "officeLocation": "ARLINGTON-1300 WILSON BLVD/Mobile",
    "preferredLanguage": null,
    "surname": "Davis",
    "userPrincipalName": "patrickdavis@microsoft.com",
    "id": "7b209f8f-5217-4a18-9bcb-17de3db15302"
}


In [19]:
#define the tools that we need to use in this case weather tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for only Tokyo and Paris.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Paris",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_lat_long_weather",
            "description": "Accurately get the weather and temperature for any location in the United States given a latitude and longitude.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "The latitude of the location",
                    },
                    "longitude": {
                        "type": "number",
                        "description": "The longitude of the location",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "user_information",
            "description": "gets information about the user such as email, work location, role, and phone number.",
            "parameters": {},
        },
    }
]

available_functions = {
                "get_current_weather": get_current_weather,
                "get_lat_long_weather": get_lat_long_weather,
                "user_information": user_information,
                } 


#System Prompt
messages = [{"role": "system", "content": "Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure."}]
print("Azure OpenAI client created and system prompt loaded.")


print("Function and tools defined")

Azure OpenAI client created and system prompt loaded.
Function and tools defined


In [20]:
#Example Questions
#Who are you?
#What is the temperature in paris?
#What is the temperature in Washington DC?
# What is the temperature in cities of Paris, France, Washington, DC and St Louis, MO?
# what is current temperature at my work address?
message = input("Your question: ")
messages.append({"role": "user", "content": message})

print(messages)

[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'what is current temperature at my work address?'}]


In [29]:
print("Starting Messages:")
print(messages)
response = client.chat.completions.create(
                model=model, 
                messages=messages,
                tools=tools,
                tool_choice="auto",
            )

print("\n\n###########Full Response###########")
print(response)
print("\n\n###########Response to User###########")
print(response.choices[0].message.content)
print("\ntools:")
print(response.choices[0].message.tool_calls)
print("##############################")
#print("\n\n###########Filter###########")
#print(json.dumps(response.choices[0].content_filter_results, indent=4))
#print("############################")


if (response.choices[0].message.tool_calls):
    messages.append({"content": response.choices[0].message.content,
                "role": response.choices[0].message.role,
                "tool_calls":response.choices[0].message.tool_calls
                })
else:
        messages.append({"content": response.choices[0].message.content,
                "role": response.choices[0].message.role })

print("\n\n###########Ending Messages###########")
print(messages)

Starting Messages:
[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'what is current temperature at my work address?'}, {'content': None, 'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_nEz1MsUlfdamBEtVLL1g6Vks', function=Function(arguments='{}', name='user_information'), type='function')]}, {'tool_call_id': 'call_nEz1MsUlfdamBEtVLL1g6Vks', 'role': 'tool', 'name': 'user_information', 'content': '{\n    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",\n    "businessPhones": [],\n    "displayName": "Patrick Davis",\n    "givenName": "Patrick",\n    "jobTitle": "SR TECHNICAL SPECIALIST",\n    "mail": "patrickdavis@microsoft.com",\n    "mobilePhone": null,\n    "officeLocation": "ARLINGTON-1300 WILSON BLVD/Mobile",\n    "preferredLanguage": null,\n    "surname": "Davis",\n    "userPrincipalName": "patrickda

In [26]:
tool_calls = response.choices[0].message.tool_calls

print("###########Tool Calls###########")
print(tool_calls)

###########Tool Calls###########
[ChatCompletionMessageToolCall(id='call_qHZ9EHeJsTHUYKzZ1BJTDjCU', function=Function(arguments='{\n  "latitude": 38.89511,\n  "longitude": -77.03637\n}', name='get_lat_long_weather'), type='function')]


In [27]:
# Select the tool to use 
# iterate though all the tool requests 
# in this example there may be more if multiple cities are requested. 
print("max tool request index: ", len(tool_calls) - 1)
#toolItr = 0
toolItr = input("Enter the tool index to use(Start with 0): ")

tool_call = tool_calls[int(toolItr)]


# Call the function
function_name  = tool_call.function.name
function_to_call = available_functions[function_name]
function_args  = json.loads(tool_call.function.arguments)
function_response = function_to_call(obj=function_args)

print("\n\n###########Function Response###########")
print(json.dumps(function_response, indent=4))



max tool request index:  0


###########Function Response###########
"{\"latitude\": \"38.89511\", \"longitude\": \"-77.03637\", \"temperature\": 55, \"unit\": \"fahrenheit\", \"detailedForecast\": \"Sunny, with a high near 55. Northwest wind 21 to 24 mph, with gusts as high as 45 mph.\", \"name\": \"This Afternoon\"}"


In [28]:
#Append the function response to the messages

messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

print("\n\n###########Ending Messages###########")
print(messages)



###########Ending Messages###########
[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'what is current temperature at my work address?'}, {'content': None, 'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_nEz1MsUlfdamBEtVLL1g6Vks', function=Function(arguments='{}', name='user_information'), type='function')]}, {'tool_call_id': 'call_nEz1MsUlfdamBEtVLL1g6Vks', 'role': 'tool', 'name': 'user_information', 'content': '{\n    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",\n    "businessPhones": [],\n    "displayName": "Patrick Davis",\n    "givenName": "Patrick",\n    "jobTitle": "SR TECHNICAL SPECIALIST",\n    "mail": "patrickdavis@microsoft.com",\n    "mobilePhone": null,\n    "officeLocation": "ARLINGTON-1300 WILSON BLVD/Mobile",\n    "preferredLanguage": null,\n    "surname": "Davis",\n    "userPrinc